In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
import time

IMAGE_SIZE = 128
MODEL_PATH = 'fingers_detection.keras'

def process_image(img):
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = img.astype(np.float32) / 255.0
    img = np.reshape(img, (IMAGE_SIZE, IMAGE_SIZE, 1))
    return img

print("Loading model...")
model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded successfully!")

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()

ret, frame = cap.read()
if not ret:
    print("Error: Could not read from webcam")
    exit()

height, width = frame.shape[:2]
square_size = min(width, height) // 2
top_left_x = (width - square_size) // 2
top_left_y = (height - square_size) // 2
bottom_right_x = top_left_x + square_size
bottom_right_y = top_left_y + square_size

print("Press 'q' to quit")

prev_time = time.time()
fps = 0
fps_counter = 0
fps_update_interval = 10

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read from webcam")
            break
            
        frame = cv2.flip(frame, 1)
        
        cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 2)
        
        hand_region = frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
        
        if hand_region.size > 0:
            gray_hand = cv2.cvtColor(hand_region, cv2.COLOR_BGR2GRAY)            
            processed_hand = process_image(gray_hand)
            img_batch = np.expand_dims(processed_hand, axis=0)
            
            predictions = model.predict(img_batch, verbose=0)
            predicted_class = np.argmax(predictions)
            prediction_confidence = np.max(predictions)
            
            text = f"Finger: {predicted_class} (Conf: {prediction_confidence:.2f})"
            cv2.putText(frame, text, (top_left_x, top_left_y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            fps_counter += 1
            if fps_counter >= fps_update_interval:
                current_time = time.time()
                fps = fps_update_interval / (current_time - prev_time)
                prev_time = current_time
                fps_counter = 0
                
            cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imshow('Finger Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Application closed")

Loading model...
Model loaded successfully!
Press 'q' to quit
Application closed
